In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [910 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 k

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-22 01:05:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-08-22 01:05:51 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://myopia.s3.amazonaws.com/Copy_of_myopia.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("Copy_of_myopia.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+---+---------+------+---+------+------+-----+-----+-----+-----+-------+------+------+-------+----+---------+-----+-----+
| ID|STUDYYEAR|MYOPIC|AGE|GENDER| SPHEQ|   AL|  ACD|   LT|  VCD|SPORTHR|READHR|COMPHR|STUDYHR|TVHR|DIOPTERHR|MOMMY|DADMY|
+---+---------+------+---+------+------+-----+-----+-----+-----+-------+------+------+-------+----+---------+-----+-----+
|  1|     1992|     1|  6|     1|-0.052|21.89| 3.69|3.498| 14.7|     45|     8|     0|      0|  10|       34|    1|    1|
|  2|     1995|     0|  6|     1| 0.608|22.38|3.702|3.392|15.29|      4|     0|     1|      1|   7|       12|    1|    1|
|  3|     1991|     0|  6|     1| 1.179|22.49|3.462|3.514|15.52|     14|     0|     2|      0|  10|       14|    0|    0|
|  4|     1990|     1|  6|     1| 0.525| 22.2|3.862|3.612|14.73|     18|    11|     0|      0|   4|       37|    0|    1|
|  5|     1995|     0|  5|     0| 0.697|23.29|3.676|3.454|16.16|     14|     0|     0|      0|   4|        4|    1|    0|
|  6|     1995|     0|  

Drop duplicates and incomplete rows

In [7]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

618
618
618


Examine the schema

In [8]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- STUDYYEAR: integer (nullable = true)
 |-- MYOPIC: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- GENDER: integer (nullable = true)
 |-- SPHEQ: double (nullable = true)
 |-- AL: double (nullable = true)
 |-- ACD: double (nullable = true)
 |-- LT: double (nullable = true)
 |-- VCD: double (nullable = true)
 |-- SPORTHR: integer (nullable = true)
 |-- READHR: integer (nullable = true)
 |-- COMPHR: integer (nullable = true)
 |-- STUDYHR: integer (nullable = true)
 |-- TVHR: integer (nullable = true)
 |-- DIOPTERHR: integer (nullable = true)
 |-- MOMMY: integer (nullable = true)
 |-- DADMY: integer (nullable = true)



Write DataFrame to RDS

In [9]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>/postgres"
config = {"user":"postgres",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [10]:
# Write DataFrame to table

df.write.jdbc(url=jdbc_url, table='myopia_data', mode=mode, properties=config)

Supervised Learning

In [12]:
#Split data into X and y
X = df.drop("MYOPIC")
y = df["MYOPIC"]

In [18]:
print (X)

DataFrame[ID: int, STUDYYEAR: int, AGE: int, GENDER: int, SPHEQ: double, AL: double, ACD: double, LT: double, VCD: double, SPORTHR: int, READHR: int, COMPHR: int, STUDYHR: int, TVHR: int, DIOPTERHR: int, MOMMY: int, DADMY: int]


In [16]:
y.head

Column<'MYOPIC[head]'>

Split the dataset

In [20]:
# Split the data into X_train, X_test, y_train, y_test
# X, y = make_classification(random_state=1)
# X = pd.DataFrame(X)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)